In [1]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
from datetime import date, timedelta
import calendar

pd.options.display.max_rows = 30

engine = create_engine("mysql+pymysql://root:@localhost:3306/stock")
con = engine.connect()
engine = create_engine("sqlite:///c:\\ruby\\portlt\\db\\development.sqlite3")
conlt = engine.connect()
engine = create_engine("sqlite:///c:\\ruby\\portmy\\db\\development.sqlite3")
conmy = engine.connect()

In [9]:
sql = '''
SELECT *
FROM buy'''
buy  = pd.read_sql(sql, con)
buy.dtypes

name         object
date         object
volbuy      float64
price       float64
volsell     float64
volbal      float64
active        int64
dividend    float64
period       object
grade        object
dtype: object

In [11]:
buy['costAmt'] = buy.volbuy * buy.price
buy

,name,date,volbuy,price,volsell,volbal,active,dividend,period,grade,costAmt
0,STA,2021-06-15,20000.0,34.75,0.0,0.0,1,1.9000,1,A1,695000.0
1,RATCH,2021-06-28,6000.0,45.75,0.0,0.0,1,3.2500,2,A4,274500.0
2,IVL,2021-03-18,3600.0,42.00,0.0,0.0,1,0.8500,3,A2,151200.0
3,KCE,2021-10-07,10000.0,77.25,0.0,0.0,1,1.0000,1,A4,772500.0
4,MCS,2016-09-20,50000.0,16.70,0.0,0.0,1,1.3700,2,A3,835000.0
...,...,...,...,...,...,...,...,...,...,...,...
26,SYNEX,2022-01-07,15000.0,29.50,0.0,0.0,1,0.5800,3,A2,442500.0
27,SCC,2021-09-20,900.0,398.00,0.0,0.0,1,9.9999,4,A2,358200.0
28,RCL,2021-11-08,4500.0,44.50,0.0,0.0,1,3.5000,3,A4,200250.0
29,RJH,2021-11-09,4500.0,33.00,0.0,0.0,1,2.5000,4,A3,148500.0


In [17]:
df_merge = pd.merge(buy, epss, on='name', how='inner')
df_merge.sort_values(['period','name'],ascending=[True,True])

,name,date,volbuy,price,volsell,volbal,active,dividend,period,grade,...,q_amt,y_amt,aq_amt,ay_amt,q_eps,y_eps,aq_eps,ay_eps,ticker_id,publish_date
2,KCE,2021-10-07,10000.0,77.25,0.0,0.0,1,1.0000,1,A4,...,700990,380581,2426284,1126787,0.60000,0.32000,2.06000,0.96000,249,2022-02-08
11,PTTGC,2021-03-17,6000.0,64.75,0.0,0.0,1,3.7500,1,A2,...,3247576,6404598,44982388,199607,0.71000,1.42000,10.01000,0.04000,385,2022-02-14
0,STA,2021-06-15,20000.0,34.75,0.0,0.0,1,1.9000,1,A1,...,1613707,5498947,15846701,9531213,1.05000,3.58000,10.32000,6.21000,481,2022-02-17
20,CPNCG,2022-02-03,10000.0,12.50,0.0,0.0,1,0.9587,2,B4,...,165405,122343,445551,501658,0.38770,0.28670,1.04430,1.17580,122,2022-02-25
3,DIF,2020-08-01,50000.0,14.70,0.0,0.0,1,1.0440,2,C4,...,4043520,4912883,12888045,13424031,0.38030,0.46210,1.21220,1.26260,140,2022-02-25
12,HREIT,2021-10-11,30000.0,8.95,0.0,0.0,1,0.6952,2,A1,...,332692,226266,892581,609075,0.39600,0.18020,1.06240,0.72500,211,2022-02-21
9,JASIF,2018-05-17,100000.0,10.00,0.0,0.0,1,0.9500,2,C4,...,2502705,2602275,8029511,9100688,0.31280,0.32530,1.00370,1.13760,234,2022-02-21
1,RATCH,2021-06-28,6000.0,45.75,0.0,0.0,1,3.2500,2,A4,...,2123212,2129472,7772022,6286678,1.46000,1.47000,5.36000,4.34000,394,2022-02-23
15,SCCC,2022-02-23,1200.0,171.00,0.0,0.0,1,9.9999,2,A2,...,1062661,896156,4247718,3680464,3.56000,3.13000,14.25000,12.35000,428,2022-02-11
10,TISCO,2021-03-23,4000.0,97.75,0.0,0.0,1,6.3000,2,B1,...,1791340,1636472,6781472,6063483,2.24000,2.04000,8.47000,7.57000,531,2022-01-14


In [8]:
sql = '''
SELECT *
FROM epss
WHERE year = 2021 AND quarter = 4'''
epss = pd.read_sql(sql, conlt)
epss.shape

(225, 14)

In [7]:
sql = '''
SELECT name, market
FROM stocks
ORDER BY name'''
my_stocks = pd.read_sql(sql, conmy)
my_stocks.shape

(376, 2)

In [8]:
filters = [
   (my_stocks.market.str.contains('SET50')),
   (my_stocks.market.str.contains('SET100')),
   (my_stocks.market.str.contains('mai'))    
]
values = [450000.0,300000.0,150000.0]

In [11]:
my_stocks["base"] = np.select(filters, values, default=150000)
my_stocks.head()

,name,market,base
0,2S,SET,150000.0
1,AAV,SET100 / SETTHSI,300000.0
2,ACE,SET100,300000.0
3,ADVANC,SET50 / SETHD / SETTHSI,450000.0
4,AEONTS,SET100,300000.0


In [13]:
df_merge = pd.merge(df,my_stocks,on='name',how='inner')
df_merge

,name,date,volbuy,price,volsell,volbal,active,dividend,period,grade,costAmt,market,base
0,STA,2021-06-15,17500.0,35.50,0.0,0.0,1,5.2500,2,A1,621250.0,SET50 / SETTHSI / SETWB,450000.0
1,RATCH,2021-06-28,6000.0,46.00,0.0,0.0,1,3.2500,3,A4,276000.0,SET50 / SETCLMV / SETHD / SETTHSI,450000.0
2,SAT,2021-07-20,15000.0,21.10,0.0,0.0,1,0.8800,4,A1,316500.0,sSET / SETTHSI,150000.0
3,IVL,2021-03-18,21000.0,42.00,0.0,0.0,1,0.8500,4,A2,882000.0,SET50 / SETTHSI,450000.0
4,LH,2021-05-10,6000.0,10.45,0.0,0.0,1,0.5500,1,C4,62700.0,SET50 / SETHD,450000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
31,BGRIM,2021-09-17,9000.0,40.00,0.0,0.0,1,0.4500,3,A2,360000.0,SET50 / SETCLMV / SETTHSI,450000.0
32,SCC,2021-09-20,600.0,405.00,0.0,0.0,1,9.9999,4,A2,243000.0,SET50 / SETCLMV / SETHD / SETTHSI,450000.0
33,RCL,2021-11-08,3600.0,43.00,0.0,0.0,1,3.5000,3,A3,154800.0,SETCLMV,150000.0
34,RJH,2021-11-09,3000.0,33.00,0.0,0.0,1,1.7000,3,A3,99000.0,sSET,150000.0


In [25]:
cols = 'name period volbuy price costAmt base pct'.split()
df_merge['pct'] = round(df_merge.costAmt/df_merge.base*100,2)
df_merge['volbuy'] = df_merge['volbuy'].astype(int)
df_merge[cols]

,name,period,volbuy,price,costAmt,base,pct
0,STA,2,17500,35.50,621250.0,450000.0,138.06
1,RATCH,3,6000,46.00,276000.0,450000.0,61.33
2,SAT,4,15000,21.10,316500.0,150000.0,211.00
3,IVL,4,21000,42.00,882000.0,450000.0,196.00
4,LH,1,6000,10.45,62700.0,450000.0,13.93
...,...,...,...,...,...,...,...
31,BGRIM,3,9000,40.00,360000.0,450000.0,80.00
32,SCC,4,600,405.00,243000.0,450000.0,54.00
33,RCL,3,3600,43.00,154800.0,150000.0,103.20
34,RJH,3,3000,33.00,99000.0,150000.0,66.00


In [26]:
mask = df.period != '2'

In [29]:
df_merge[mask].nsmallest(4, 'pct')[['name','period','volbuy','costAmt','pct']]

,name,period,volbuy,costAmt,pct
4,LH,1,6000,62700.0,13.93
9,GLOBAL,3,7500,150000.0,33.33
12,TOP,3,3000,150000.0,33.33
5,KCE,3,2000,160000.0,35.56


In [28]:
df_merge[mask].nlargest(3, 'pct')[['name','period','volbuy','costAmt','pct']]

,name,period,volbuy,costAmt,pct
2,SAT,4,15000,316500.0,211.0
3,IVL,4,21000,882000.0,196.0
25,NER,4,36000,268200.0,178.8


In [15]:
# look again at some of the data
catvars = ['period','grade']

for col in catvars:
  print(col, df[col].value_counts().sort_index(), sep="\n\n", end="\n\n\n")

period

1     1
2    12
3    16
4    11
Name: period, dtype: int64


grade

A1     8
A2    15
A3     2
A4     4
B1     1
B2     1
B3     2
B4     2
C1     2
C4     3
Name: grade, dtype: int64




In [22]:
df.loc[
    df.grade == 'A2',
['period','name','grade','costAmt']].sort_values(['period','name'],ascending=[True,True])

,period,name,grade,costAmt
29,2,BKI,A2,140000.0
12,2,TMT,A2,163500.0
14,3,BCH,A2,130200.0
39,3,BGRIM,A2,126000.0
16,3,DOHOME,A2,300000.0
5,3,IVL,A2,1008000.0
8,3,KCE,A2,291600.0
32,3,NER,A2,135000.0
23,3,PTTGC,A2,534000.0
37,3,SINGER,A2,234000.0


In [10]:
df.groupby('period').agg(
   {'costAmt':'sum','volbuy':'sum'}
).round(
   {'costAmt':2, 'volbuy':0}
)

,costAmt,volbuy
period,,
1,250800.0,24000.0
2,5321000.0,462500.0
3,4006200.0,132100.0
4,1716450.0,170900.0


In [14]:
df.replace(
   {
      'period':{'1':'Disposal','2':'Hi-Div','3':'Long Term','4':'Short Term'}
   }
).head()

,name,date,volbuy,price,volsell,volbal,active,dividend,period,grade,costAmt
0,KKP,2021-09-07,3000.0,55.00,0.0,0.0,1,3.00,Short Term,C1,165000.0
1,BCP,2021-09-06,6000.0,27.00,0.0,0.0,1,1.40,Short Term,A1,162000.0
2,STA,2021-06-15,10000.0,38.25,0.0,0.0,1,4.50,Hi-Div,A1,382500.0
3,RATCH,2021-06-28,6000.0,46.00,0.0,0.0,1,3.25,Long Term,A4,276000.0
4,SAT,2021-07-20,10000.0,21.90,0.0,0.0,1,0.88,Hi-Div,A1,219000.0


### Start of Update dividend

In [3]:
name = 'BCP'
sql = '''
SELECT * 
FROM DIVIDEND
WHERE name = "%s"'''
sql = sql % name
df = pd.read_sql(sql, con)
df

,NAME,Q4,Q3,Q2,Q1,DIVIDEND,PRICE,PERCENT,SHARES,XDATE,PAIDDATE,KIND,ACTUAL
0,BCP,0.4,0.0,1.0,0.0,1.4,0.0,0.0,6000.0,2021-09-08,2021-09-21,,1


In [3]:
shares = 350000
q4 = .1111
XDATE = '2020-05-25'
PAIDDATE = '2020-06-12'

In [4]:
sql = """
UPDATE dividend 
SET shares = %s, q4 = %s, dividend = q1+q2+q3+q4, XDATE = '%s', PAIDDATE = '%s',actual = 1
WHERE name = "%s"
"""
sql = sql % (shares, q4, XDATE, PAIDDATE, name)
rp = con.execute(sql)
rp.rowcount

1

### New dividend stock

In [7]:
sql = """
INSERT INTO dividend 
VALUES('WHA',0.0535,0.0815,0,0,0.135,0,0,00000,'2020-05-08','2020-05-25','',1)
"""
rp = con.execute(sql)
rp.rowcount

1

### Delete dividend record

In [ ]:
sql = '''
DELETE FROM dividend
WHERE name IN ("INTUCH")
'''
rp = con.execute(sql)
rp.rowcount

### Start of buy update

In [4]:
sql = """
SELECT * 
FROM buy 
WHERE active = 1
ORDER BY name"""
# sql = sql % name
df = pd.read_sql(sql, con)
df

,name,date,volbuy,price,volsell,volbal,active,dividend,period,grade
0,ADVANC,2020-04-09,600.0,190.00,0.0,0.0,1,7.1300,4,B4
1,BCP,2021-09-06,6000.0,27.00,0.0,0.0,1,0.0000,4,A1
2,BKI,2021-07-01,500.0,280.00,0.0,0.0,1,9.9999,4,A2
3,CHG,2021-07-15,50000.0,4.04,0.0,0.0,1,0.0600,3,A3
4,CPNCG,2021-05-07,20000.0,10.85,0.0,0.0,1,0.9995,2,B3
...,...,...,...,...,...,...,...,...,...,...
33,TPIPP,2020-09-11,60000.0,4.19,0.0,0.0,1,0.2700,4,B3
34,TQM,2021-05-21,1800.0,112.00,0.0,0.0,1,2.6000,3,A1
35,TU,2021-08-19,18000.0,20.80,0.0,0.0,1,0.8500,3,A2
36,TVO,2021-01-25,6000.0,33.00,0.0,0.0,1,2.1000,4,A1


In [14]:
name = 'ORI'
shares = 60000
price = 9.7

In [15]:
sql = """
UPDATE buy 
SET  period = "1",VOLBUY = %s, price = %s, active=1
WHERE name = "%s" AND active = 1"""
sql = sql % (shares, price, name)
print(sql)


UPDATE buy 
SET  period = "1",VOLBUY = 60000, price = 9.7, active=1
WHERE name = "ORI" AND active = 1


In [16]:
rp = con.execute(sql)
rp.rowcount

1

### End of Buy update

### When sold out

In [2]:
name = 'SIRI'
sql = '''
DELETE FROM buy 
WHERE name = "%s" '''
sql = sql % name
print(sql)


DELETE FROM buy 
WHERE name = "SIRI" 


In [3]:
rp = con.execute(sql)
rp.rowcount

1

### End of sold out

### Start of New buy record

In [2]:
name = "DTAC"
new_cost = 40
new_qty = 5000

In [3]:
sql = """
INSERT INTO buy 
VALUES('DTAC','2020-05-07',5000,40,0,0,1,0,'3','A2')
"""
print(sql)


INSERT INTO buy 
VALUES('DTAC','2020-05-07',5000,40,0,0,1,0,'3','A2')



In [4]:
rp = con.execute(sql)
rp.rowcount

1

### Start of change grade after quarterly eps

In [15]:
name = 'POPF'
sql = '''
SELECT * FROM buy
WHERE name = "%s"
'''
sql = sql % name
df = pd.read_sql(sql, con)
df

,name,date,volbuy,price,volsell,volbal,active,dividend,period,grade
0,POPF,2021-07-30,10000.0,11.0,0.0,0.0,1,1.001,4,C1


In [14]:
sql = '''
UPDATE buy
SET period = "4", grade = "C1"
WHERE name = "%s"
'''
sql = sql % name
rp = con.execute(sql)
rp.rowcount

1

### End of change grade after quarterly eps

### Change period according to grade

In [ ]:
sql = '''
UPDATE buy
SET period = "2"
WHERE grade =  "B1"'''
print(sql)

In [ ]:
sql = '''
UPDATE buy
SET period = "1"
WHERE grade =  "C4"'''
print(sql)

In [ ]:
sql = '''
UPDATE buy
SET period = "3"
WHERE grade =  "A3"'''
print(sql)

In [ ]:
sql = """
UPDATE buy
SET period = "4"
WHERE name IN ("IVL","PTTGC","SAT")"""
print(sql)

In [ ]:
sql = """
UPDATE buy
SET period = "2"
WHERE name IN ("JASIF")"""
print(sql)

In [ ]:
sql = '''
UPDATE buy
SET period = "2"
WHERE grade =  "A1"'''
print(sql)

In [ ]:
rp = con.execute(sql)
rp.rowcount

### End of Change period according to grade

### Change grade of specific stock

In [ ]:
sql = """
UPDATE buy
SET period = "2"
WHERE name IN ('ANAN','LH','MCS','ORI')"""
print(sql)

In [ ]:
sql = """
UPDATE buy
SET period = "3"
WHERE name IN ('AMATA')"""
print(sql)

In [ ]:
rp = con.execute(sql)
rp.rowcount

### End of Change grade of specific stock

In [ ]:
sql = """
INSERT INTO buy
VALUES('GC','2018-02-28',10000,5.3,)"""

In [ ]:
sql = '''
UPDATE buy
SET period = "4"
WHERE grade <>  "A4"'''
print(sql)

In [ ]:
rp = con.execute(sql)
rp.rowcount

In [ ]:
rp = con.execute(sql)
rp.rowcount

In [ ]:
sql = '''
UPDATE buy
SET period = "1"
WHERE substr(grade,1,1) <>  "A"'''
print(sql)

In [ ]:
rp = con.execute(sql)
rp.rowcount

In [ ]:
rp = con.execute(sql)
rp.rowcount

### Individual record change

In [ ]:
sql = """
UPDATE buy
SET period = "2"
WHERE name IN ('GC','LH','TISCO','MCS','BR')"""
print(sql)

In [ ]:
rp = con.execute(sql)
rp.rowcount

### Par value change

In [ ]:
sql = """
SELECT *
FROM price
LIMIT 1"""
prices = pd.read_sql(sql, con)
prices.dtypes

In [ ]:
name = "PTT"
effective_date = "2018-04-24"

In [ ]:
sql = """
UPDATE price
SET price = price*10, maxp = maxp*10, minp = minp*10, opnp = opnp*10,
qty = qty/10
WHERE name = '%s' AND date < '%s'
"""
sql = sql % (name, effective_date)
print(sql)

In [ ]:
rp = con.execute(sql)
rp.rowcount

In [ ]:
sql = """
SELECT *
FROM buy
LIMIT 1"""
buys = pd.read_sql(sql, con)
buys.dtypes

In [ ]:
sql = """
UPDATE buy
SET volbuy = 4000, price = 50.8
WHERE name = '%s' AND active = 1
"""
sql = sql % (name)
print(sql)

In [ ]:
rp = con.execute(sql)
rp.rowcount

### Name change

In [8]:
old_name = "EPCO"
new_name = "EP"

In [9]:
sql = """
SELECT *
FROM stockname
WHERE name = '%s'
"""
sql = sql % (old_name)
stock = pd.read_sql(sql, con)
stock

,name,category,sector,inport,onwatch,hd,top100
0,EPCO,,MEDIA,0,0,0,0


In [10]:
sql = """
UPDATE stockname
SET name = '%s'
WHERE name = '%s'
"""
sql = sql % (new_name, old_name)
print(sql)


UPDATE stockname
SET name = 'EP'
WHERE name = 'EPCO'



In [11]:
rp = con.execute(sql)
rp.rowcount

1

In [12]:
sql = """
SELECT *
FROM price
WHERE name = '%s'
"""
sql = sql % (old_name)
price = pd.read_sql(sql, con)
price.shape

(1533, 7)

In [13]:
sql = """
UPDATE price
SET name = '%s'
WHERE name = '%s'
"""
sql = sql % (new_name, old_name)
print(sql)


UPDATE price
SET name = 'EP'
WHERE name = 'EPCO'



In [14]:
rp = con.execute(sql)
rp.rowcount

1533

In [3]:
sql = '''
SELECT * FROM price WHERE name = 'ASK' ORDER BY date DESC'''
price = pd.read_sql(sql, con)
price

,name,date,price,maxp,minp,qty,opnp
0,ASK,2020-06-05,19.3,19.3,19.1,1180345,19.2
1,ASK,2020-06-04,19.1,19.4,19.0,984900,19.0
2,ASK,2020-06-02,18.8,19.1,18.7,488089,18.7
3,ASK,2020-06-01,18.7,18.9,18.5,1133238,18.5
4,ASK,2020-05-29,18.4,18.6,18.0,617730,18.3
5,ASK,2020-05-28,18.3,18.7,18.3,940551,18.5
6,ASK,2020-05-27,18.4,18.7,18.1,1498725,18.7
7,ASK,2020-05-26,18.4,18.5,18.0,1635709,18.0
8,ASK,2020-05-25,18.0,18.2,17.7,949532,17.9
9,ASK,2020-05-22,17.9,17.9,17.5,1657930,17.9


In [2]:
sql = '''
DELETE FROM price WHERE date = "2021-12-14"'''
rp = con.execute(sql)
rp.rowcount

0

In [11]:
sql = """
INSERT INTO buy (name, date, volbuy, price, volsell, volbal, active, dividend, period, grade)
VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
"""
print(sql)


INSERT INTO buy (name, date, volbuy, price, volsell, volbal, active, dividend, period, grade)
VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)



In [12]:
rcd = ['TSE', '2022-01-21', 40000.0, 2.60, 0, 0, 1, 0.10, '3', 'A4']
rcd

['TSE', '2022-01-21', 40000.0, 2.6, 0, 0, 1, 0.1, '3', 'A4']

In [15]:
rp = con.execute(sql, rcd)
rp.rowcount

1

In [20]:
sqlDel = '''
DELETE
FROM buy
WHERE name = "TSE"'''
rp = con.execute(sqlDel)
rp.rowcount

5